In [1]:
import os
import sys
import time
import socket
import random
import ok


# file_dir = os.path.dirname(os.path.realpath(__file__))
# parent_dir = os.path.dirname(file_dir)
# sys.path.append(parent_dir)
dev_info = ok.okTDeviceInfo()
print("------ PipeTest Application fot Python API ----------")
print("Front Panel API version: {}".format(ok.okCFrontPanel_GetAPIVersionString()))
dev = ok.okCFrontPanel()
error_code = dev.OpenBySerial("")
if error_code != 0:
    print('Could not open the device.')
    print(error_code)
    sys.exit(1)



def initialize_FPGA(dev: ok.okCFrontPanel, bit_file_path):
    dev.GetDeviceInfo(dev_info)
    print("Found device: {}".format(dev_info.productName))
    dev.LoadDefaultPLLConfiguration()
    print('Device firmware version: {}.{}'.format(dev_info.deviceMajorVersion, dev_info.deviceMinorVersion))
    print('Device serial number: {}'.format(dev_info.serialNumber))
    print('Device ID: {}'.format(dev_info.productID))
    print(f"loading {bit_file_path} as bit file")
    if (dev.ConfigureFPGA(bit_file_path) != ok.okCFrontPanel.NoError):
        print("Fpga configuration failed.")
        return False
    print("success configuration")
# initialize_FPGA(dev,"bitstreams/main_v7_clock_port_regfileonly.bit")
# initialize_FPGA(dev,"bitstreams/main_v8_fifo.bit")
# initialize_FPGA(dev,"bitstreams/main_v6.bit")
# initialize_FPGA(dev,"bitstreams/main_v13.bit")
# initialize_FPGA(dev,"bitstreams/main_v13_v6.bit")
# initialize_FPGA(dev,"bitstreams/main_v13_v6_v4.bit")
# initialize_FPGA(dev,"bitstreams/main_v13_fixed.bit")
initialize_FPGA(dev,"bitstreams/main.bit")

# ep_addr(8'h14), .ep_datain(ep20wireout));
# ep_addr(8'h15), .ep_datain(ep21wireout));
# ep_addr(8'h16), .ep_datain(ep22wireout));
# ep_addr(8'h18), .ep_datain(ep24wireout_readout));
# ep_addr(8'ha0), .ep_read(FIFO_read_from), .ep_blo



ep00wirein_address =  0x00
ep01wirein_address =  0x01
ep02wirein_address =  0x02
ep03wirein_address =  0x03
ep04wirein_address =  0x04
ep05wirein_address =  0x05
ep06wirein_address =  0x06
ep07wirein_address =  0x07
ep08wirein_address =  0x08
ep09wirein_address =  0x09
ep0awirein_address =  0x0A
ep0bwirein_address =  0x0B
ep0cwirein_address =  0x0C
ep0dwirein_address =  0x0D
ep0ewirein_address =  0x0E
ep0fwirein_address =  0x0F
ep10wirein_address =  0x10
ep11wirein_address =  0x11
ep12wirein_address =  0x12
ep13wirein_address =  0x13
ep14wirein_address =  0x14
ep15wirein_address =  0x15
ep16wirein_address =  0x16
ep17wirein_address =  0x17
ep18wirein_address =  0x18
ep19wirein_address =  0x19
ep1awirein_address =  0x1A
ep1bwirein_address =  0x1B              
ep1cwirein_address =  0x1C
ep1dwirein_address =  0x1D
ep1ewirein_address =  0x1E
ep1fwirein_address =  0x1F

# Trigger in endpoints
ep40trigin_address =  0x40
ep41trigin_address =  0x41

# wire out endpoints
ep20wireout_address = 0x20
ep21wireout_address = 0x21
ep22wireout_address = 0x22
ep23wireout_address = 0x23
ep24wireout_address = 0x24
ep25wireout_address = 0x25
ep26wireout_address = 0x26
ep27wireout_address = 0x27
ep28wireout_address = 0x28
ep29wireout_address = 0x29
ep2awireout_address = 0x2A
ep2bwireout_address = 0x2B
ep2cwireout_address = 0x2C


ep3dwireout_address = 0x3D
ep3ewireout_address = 0x3E
ep3fwireout_address = 0x3F
# ep22wireout_address = 0x16
# ep24wireout_address = 0x18
epPipeOut_address   = 0xA0

# functions
def write_wr_in(dev,wr_addr,data,mask = 0xffffffff):
    dev.SetWireInValue(wr_addr, data,mask)
    dev.UpdateWireIns()  
def reset_sig(dev,ep_address):
    write_wr_in(dev,ep_address,0x00000001,mask=0x00000001)
    time.sleep(0.1)
    write_wr_in(dev,ep_address,0x00000000,mask=0x00000001)
    time.sleep(0.1)
def write_length(dev,ep05wirein_address,length):
    write_wr_in(dev,ep05wirein_address,length)
    time.sleep(0.1)
def trigger_in(dev,trigin_address,index):
    dev.ActivateTriggerIn(trigin_address, index)
def trigger_spi(dev,trigin_address):
    dev.ActivateTriggerIn(trigin_address, 0)
    time.sleep(0.1)
def read_wire_out(dev,address):
    dev.UpdateWireOuts()
    ep20_val = dev.GetWireOutValue(address)
    return ep20_val
    # out_value = (ep20_val >> 8) & 0x7F  
    # out_debug = ep20_val & 0x7F
    # # print(f"Extracted ep{address}_val value: {ep20_val}")
    # # print(f"Extracted out value: {out_value}")
    # print(f"Extracted out_debug value: {out_debug}")
    # # return out_value, out_debug
def read_wire_out_print(dev, address, mode='dec'):
    dev.UpdateWireOuts()
    ep_val = dev.GetWireOutValue(address)
    if mode == 'hex':
        print(f"WireOut {hex(address)}: 0x{ep_val:04X}")
    else:  # default to decimal
        print(f"WireOut {hex(address)}: {ep_val}")
    return ep_val
def write_usb3_blocksize(dev,usb3_blocksize=0x00000000):
    write_wr_in(dev,ep1fwirein_address,usb3_blocksize)
    trigger_in(dev,ep40trigin_address,index=9)
def write_ddr_blocksize(dev,ddr_blocksize=0x00000000):
    write_wr_in(dev,ep1fwirein_address,ddr_blocksize)
    trigger_in(dev,ep40trigin_address,index=10)
def write_dataclk_MD(dev,Hex_00_M_D=0x00002A19):
    write_wr_in(dev,ep03wirein_address,Hex_00_M_D)
    trigger_in(dev,ep40trigin_address,index=0)      # MMCM_prog_trigger
def enable_streams(dev, enable_hex=0x00000000):
    write_wr_in(dev,ep14wirein_address,enable_hex) 
def write_DSP_settle_en(dev,Hex=0x00000000):
    write_wr_in(dev,ep1fwirein_address,Hex)
    trigger_in(dev,ep40trigin_address,index=6)  
    # trigger_in(dev,ep40trigin_address,index=7)      # MMCM_prog_trigger
def write_DAC_set_zero(dev):
    write_wr_in(dev,ep1fwirein_address,0x00000000)
    for i in range(32):
        trigger_in(dev,ep40trigin_address,index=i)  
def write_delay(dev,delay=0x00000001):
    write_wr_in(dev,ep04wirein_address,delay,mask=0x0000000f)
def write_max_time_samples(dev,time_step=0x0000001):
    print(f"Writing time step: {time_step}")
    write_wr_in(dev,ep01wirein_address,time_step)
def dac_stuff(dev):
    write_wr_in(dev,ep08wirein_address,0x00000000) 
    write_wr_in(dev,ep09wirein_address,0x00000000) 
    write_wr_in(dev,ep0awirein_address,0x00000000) 
    write_wr_in(dev,ep0bwirein_address,0x00000000) 
    write_wr_in(dev,ep0cwirein_address,0x00000000) 
    write_wr_in(    dev,ep15wirein_address,0x00000000) 
    write_wr_in(    dev,ep16wirein_address,0x00000000) 
    write_wr_in(    dev,ep17wirein_address,0x00000000) 
    write_wr_in(    dev,ep18wirein_address,0x00000000) 
    write_wr_in(    dev,ep19wirein_address,0x00000000) 
    write_wr_in(    dev,ep1awirein_address,0x00000000) 
    write_wr_in(    dev,ep1bwirein_address,0x00000000) 
    write_wr_in(    dev,ep1cwirein_address,0x00000000) 
    write_wr_in(    dev,ep1dwirein_address,0x00000000) 
    write_wr_in(    dev,ep1ewirein_address,0x00000000) 
def mux_emulator_type(dev,external_emulator=False,hardcoded=False):
    value_send = 0x00000000
    if external_emulator:
        value_send = value_send | 0x00000002
    if hardcoded:
        value_send = value_send | 0x00000001
    write_wr_in(dev,ep0fwirein_address,value_send)
    # trigger_in(dev,ep40trigin_address,index=10)
def rst_emulator(dev):
    write_wr_in(dev,ep0fwirein_address,0x00000004)

# write_out wires status
# read_wire_out_print(dev, ep20wireout_address, mode='hex')
# read_wire_out_print(dev, ep21wireout_address, mode='hex')
# read_wire_out_print(dev, ep22wireout_address, mode='hex')
# read_wire_out_print(dev, ep23wireout_address, mode='hex')
# read_wire_out_print(dev, ep24wireout_address, mode='hex')
# read_wire_out_print(dev, ep25wireout_address, mode='hex')
# read_wire_out_print(dev, ep3ewireout_address, mode='hex')
# read_wire_out_print(dev, ep3ewireout_address, mode='dec')
read_wire_out_print(dev, ep3fwireout_address, mode='hex')
read_wire_out_print(dev, ep3fwireout_address, mode='dec')


print("FPGA is ready, starting INTAN configuration...")
#Trigger_in40 
reset_sig(              dev,    ep00wirein_address)
print("Reset done")
write_usb3_blocksize(   dev,    usb3_blocksize=0x00000000)
write_ddr_blocksize(    dev,    ddr_blocksize =0x00000000) 
write_dataclk_MD(       dev,    Hex_00_M_D    =0x00002A19) #
while (read_wire_out_print(dev, ep24wireout_address, mode='hex') != 0x00000003):
    print("Waiting for FPGA to be ready...")
    # time.sleep(0.5)


# DAC off, not used in this version
dac_stuff(dev)
write_DSP_settle_en(dev)
write_DAC_set_zero(dev)



print("Enabling streams...")
enable_streams( dev, enable_hex=0x0000000F)
# write_delay(dev,delay=0x00000001)
write_delay(dev,delay=0x00000002)
max_time_samples = 0x00000010
# max_time_samples = 0x00000004
write_max_time_samples(dev,time_step=max_time_samples)
print(f"FPGA is ready, expectiing {max_time_samples} samples")
# mux_emulator_type(dev,external_emulator=True,hardcoded=False)
mux_emulator_type(dev,external_emulator=False,hardcoded=False)
# mux_emulator_type(dev,external_emulator=False,hardcoded=True)
read_wire_out_print(dev, ep20wireout_address, mode='dec')

write_wr_in(dev,ep11wirein_address,0x00000001,mask=0x00000001)
write_wr_in(dev,ep11wirein_address,0x00000000,mask=0x00000001)






------ PipeTest Application fot Python API ----------
Front Panel API version: 5.3.5
Could not open the device.
-1


SystemExit: 1

C:\Users\Turog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def build_ep13wirein(data: int, turn_on: int) -> int:
    value = (turn_on << 7) | data
    print(f"Building ep10wirein: data={data}, reset={turn_on} -> 0x{value:04X}")
    return value


def decode_ep2fwireout(value,index):
    assert 0 <= value < (1 << 32), "Input must be a 32-bit integer"

    # Upper and lower 16-bit words
    data_out1 = (value >> 16) & 0xFFFF
    data_out0 = value & 0xFFFF

    # Display nicely
    print(f"Decoding ep2fwireout: value=0x{value:08X} (index={index})")
    print(f"Raw Hex:     0x{value:08X}")
    print(f"Raw Binary:  {'_'.join(f'{value:032b}'[i:i+4] for i in range(0, 32, 4))}")
    print(f"Decoded Fields:")
    print(f"  data_out1: 0x{data_out1:04X} ({data_out1})")
    print(f"  data_out0: 0x{data_out0:04X} ({data_out0})")
    return data_out1, data_out0

ep2fwireout_address = 0x2f
def get_dual_value(index):
    build_ep13wirein = build_ep13wirein(index, turn_on=1)
    write_wr_in(dev,ep13wirein_address,build_ep13wirein)
    dev.UpdateWireOuts()
    value_out = dev.GetWireOutValue(ep2fwireout_address)
    out, out_debug = decode_ep2fwireout(value_out)
    build_ep13wirein = build_ep13wirein(0, turn_on=0)



In [ ]:
def build_ep10wirein(data: int, reset: int, wr_addr: int, write_enable: int) -> int:
    assert 0 <= data < (1 << 7), "data must be 7-bit"
    assert reset in (0, 1), "reset must be 0 or 1"
    assert 0 <= wr_addr < (1 << 7), "wr_addr must be 7-bit"
    assert write_enable in (0, 1), "write_enable must be 0 or 1"
    value = (write_enable << 15) | (wr_addr << 8) | (reset << 7) | data
    print(f"Building ep10wirein: data={data}, reset={reset}, wr_addr={wr_addr}, write_enable={write_enable} -> 0x{value:04X}")
    return value

def print_ep10wirein(ep10_value: int):
    print(f"Hex:     0x{ep10_value:04X}")
    bin_str = f"{ep10_value:016b}"  # 16-bit binary string
    grouped_bin = "_".join([bin_str[i:i+4] for i in range(0, 16, 4)])
    print(f"Binary:  {grouped_bin}")

def decode_ep26wireout(value: int):
    assert 0 <= value < (1 << 32), "Input must be a 32-bit integer"
    out = (value >> 14) & 0xFF
    out_debug = value & 0xFF  # Lower 13 bits
    print(f"Raw Hex:     0x{value:08X}")
    print(f"Raw Binary:  {'_'.join(f'{value:032b}'[i:i+4] for i in range(0, 32, 4))}")
    print(f"Decoded Fields:")
    # print(f"  out:        {out}")
    print(f"  out_debug:  0x{out_debug:04X} ({out_debug})")
    return out, out_debug

def write_to_regfile(dev, send_out_index, address_requested,DEBUG=False):
    ep10_value_i   = build_ep10wirein(data=address_requested, reset=0, wr_addr=send_out_index, write_enable=0)
    write_wr_in(dev,ep10wirein_address,ep10_value_i)
    if DEBUG:
        print(f"write_to_regfile: address_requested = {address_requested}")
        print(f"write_to_regfile: send_out_index = {send_out_index}")
        print(f"write_to_regfile: ep10_value_i = {ep10_value_i}")
    ep10_value_i   = build_ep10wirein(data=address_requested, reset=0, wr_addr=send_out_index, write_enable=1)
    write_wr_in(dev,ep10wirein_address,ep10_value_i)
    if DEBUG:
        print(f"write_to_regfile: ep10_value_i = {ep10_value_i}")
    ep10_value_i   = build_ep10wirein(data=address_requested, reset=0, wr_addr=send_out_index, write_enable=0)
    write_wr_in(dev,ep10wirein_address,ep10_value_i)
    if DEBUG:
        print(f"write_to_regfile: ep10_value_i = {ep10_value_i}")

def read_regfile(dev, send_out_index,DEBUG=False):
    ep10_value_i   = build_ep10wirein(data=0, reset=0, wr_addr=send_out_index, write_enable=0)
    write_wr_in(dev,ep10wirein_address,ep10_value_i)
    dev.UpdateWireOuts()

    address_out = dev.GetWireOutValue(ep26wireout_address)
    print(f"wr_addr: {send_out_index}")
    out, out_debug = decode_ep26wireout(address_out)
    return send_out_index,out_debug

def return_fifo_size(dev):
    dev.UpdateWireOuts()
    num_words_in_FIFO = dev.GetWireOutValue(ep27wireout_address)
    print(f"FIFO size: {num_words_in_FIFO}")
    # return num_words_in_FIFO


def max_readout(dev,max=0x0000007F):
    write_wr_in(dev,ep12wirein_address,max)
max_readout(dev,max=0x00000005)
# max_readout(dev,max=0x00000005)
for i in range(0x00, 0x20):
    # write_to_regfile(dev, send_out_index=i, address_requested=i,DEBUG=True)
    
    address,out_debug = read_regfile(dev, send_out_index=i,DEBUG=False)
    print(f"Address: {address}, Out Debug: {out_debug}")
    # address,out_debug = read_regfile(dev, send_out_index=i,DEBUG=True)
    print("")
    # out_debug = read_regfile(dev, send_out_index=0x00,DEBUG=True)

return_fifo_size(dev)




In [ ]:
dev.ActivateTriggerIn(ep41trigin_address, 0)
# ep41trigin
print(f" Final EP00 wireout value: {read_wire_out_print(dev, ep00wirein_address, mode='hex')}")
print("FPGA configuration done, starting data acquisition...")
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')

In [ ]:
return_fifo_size(dev)


def return_fifo_size(dev):
    dev.UpdateWireOuts()
    num_words_in_FIFO = dev.GetWireOutValue(ep27wireout_address)
    print(f"FIFO size: {num_words_in_FIFO}")
    # return num_words_in_FIFO



In [ ]:
# ep28wireout_address = 0x28
read_wire_out_print(dev, ep28wireout_address, mode='dec')
read_wire_out_print(dev, ep29wireout_address, mode='dec')
read_wire_out_print(dev, ep29wireout_address, mode='hex')
read_wire_out_print(dev, ep2awireout_address, mode='dec')
read_wire_out_print(dev, ep2bwireout_address, mode='dec')
read_wire_out_print(dev, ep2cwireout_address, mode='dec')
read_wire_out_print(dev, 0x2D, mode='dec')
read_wire_out_print(dev, 0x2E, mode='dec')



# def max_readout(dev,max=0x0000007F):
#     write_wr_in(dev,ep12wirein_address,max)
# max_readout(dev,max=0x00000005)

print("Data acquisition done.")

In [ ]:
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")

In [ ]:
import time

ep_addr = 0xA0  # Replace with your actual PipeOut endpoint address
ep20wireout_address = 0x20  # Replace with your actual WireOut address
chunk_size = 128  # Number of bytes to read per chunk
output_file = 'output_emulator_v14.txt'  # Output file path
counts_max = 156 * (max_time_samples-1)*2
counts  = 0
with open(output_file, 'w') as f:
    while True:
        # Update and read the WireOut value to determine bytes left
        dev.UpdateWireOuts()
        bytes_left = dev.GetWireOutValue(ep20wireout_address)
        print(f"WireOut {hex(ep20wireout_address)}: {bytes_left} bytes left")

        # Check if the remaining bytes are less than the chunk size
        if bytes_left < chunk_size:
            print(f"Final read: only {bytes_left} bytes left, which is less than the chunk size ({chunk_size} bytes).")
            break

        print(f"Reading {chunk_size} bytes from PipeOut endpoint {hex(ep_addr)}...")
        data = bytearray(chunk_size)
        ret = dev.ReadFromBlockPipeOut(ep_addr, chunk_size, data)

        if ret < 0:
            print(f"Error reading from Block Pipe Out. Error code: {ret}")
            break

        print("Data acquisition done.")

        for i in range(0, len(data), 4):
            chunk = data[i:i+4]
            if len(chunk) < 4:
                print("Incomplete 4-byte chunk encountered; skipping.")
                continue
            value = int.from_bytes(chunk, byteorder='little')  # Adjust 'little' or 'big' as needed
            # value = int.from_bytes(chunk, byteorder='big')  # Adjust 'little' or 'big' as needed
            line = f"0x{value:08X}"
            # line = f"Index {i//4:05d}: 0x{value:08X}"
            # print(line)
            if counts < counts_max: 
                f.write(line + '\n')
            counts += 1

        # time.sleep(0.1)  # Optional: add delay between reads


In [ ]:
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")

In [ ]:
set_of_32 = 2
length = 0x3F
ep_addr = epPipeOut_address 
# num_bytes = 32*4*set_of_32  
num_bytes = 1024  

print(f"Reading {num_bytes} bytes from PipeOut endpoint {hex(ep_addr)}...")
data = bytearray(num_bytes)
ret = dev.ReadFromBlockPipeOut(ep_addr, num_bytes, data)
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")
if ret < 0:
    print("Error reading from Block Pipe Out. Error code:", ret,num_bytes,data)
else:
    print(data)

    for i in range(0, len(data), 4):
        chunk = data[i:i+4]
        if len(chunk) < 4:
            break  # Skip incomplete chunks at the end
        value = int.from_bytes(chunk, byteorder='little')  # Use 'big' if your data is big-endian
        print(f"Index {i//4:05d}: 0x{value:08X}")






In [ ]:
import time

ep_addr = 0xA1  # Replace with your actual PipeOut endpoint address
ep20wireout_address = 0x27  # Replace with your actual WireOut address
chunk_size = 16  # Number of bytes to read per chunk
output_file = 'output_emulator_v15.txt'  # Output file path
counts_max = 156 * (max_time_samples-1)*2
counts  = 0
with open(output_file, 'w') as f:
    while True:
        # Update and read the WireOut value to determine bytes left
        dev.UpdateWireOuts()
        bytes_left = dev.GetWireOutValue(ep20wireout_address)
        print(f"WireOut {hex(ep20wireout_address)}: {bytes_left} bytes left")

        # Check if the remaining bytes are less than the chunk size
        if bytes_left < chunk_size:
            print(f"Final read: only {bytes_left} bytes left, which is less than the chunk size ({chunk_size} bytes).")
            break

        print(f"Reading {chunk_size} bytes from PipeOut endpoint {hex(ep_addr)}...")
        data = bytearray(chunk_size)
        
        ret = dev.ReadFromBlockPipeOut(ep_addr, chunk_size, data)

        if ret < 0:
            print(f"Error reading from Block Pipe Out. Error code: {ret}")
            break

        print("Data acquisition done.")
        print(data)

        for i in range(0, len(data), 4):
            chunk = data[i:i+4]
            if len(chunk) < 4:
                print("Incomplete 4-byte chunk encountered; skipping.")
                continue
            value = int.from_bytes(chunk, byteorder='little')  # Adjust 'little' or 'big' as needed
            # value = int.from_bytes(chunk, byteorder='big')  # Adjust 'little' or 'big' as needed
            line = f"0x{value:08X}"
            # line = f"Index {i//4:05d}: 0x{value:08X}"
            print(line)
            if counts < counts_max: 
                f.write(line + '\n')
            counts += 1

        # time.sleep(0.1)  # Optional: add delay between reads


In [ ]:
print(data)

In [ ]:

set_of_32 = 2
length = 0x3F
ep_addr = epPipeOut_address 
# num_bytes = 32*4*set_of_32  
num_bytes = 1024  

print(f"Reading {num_bytes} bytes from PipeOut endpoint {hex(ep_addr)}...")
data = bytearray(num_bytes)
ret = dev.ReadFromBlockPipeOut(ep_addr, num_bytes, data)
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")
if ret < 0:
    print("Error reading from Block Pipe Out. Error code:", ret,num_bytes,data)
else:
    print(data)

    for i in range(0, len(data), 4):
        chunk = data[i:i+4]
        if len(chunk) < 4:
            break  # Skip incomplete chunks at the end
        value = int.from_bytes(chunk, byteorder='little')  # Use 'big' if your data is big-endian
        print(f"Index {i//4:05d}: 0x{value:08X}")




